In [1]:
pip install mani_skill

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 MB 19.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 623.8/623.8 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 64.3 MB/s eta 0:00:0000:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 MB 28.1 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.8/113.8 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 704.8/704.8 kB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 638.2/638.2 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 2.4 MB/s eta 0:00:00
  Attempting uni

In [2]:
from tqdm.notebook import tqdm

import torch
import gymnasium as gym
from ppo_rgb import Agent

import mani_skill.envs
from mani_skill.utils.wrappers.flatten import FlattenRGBDObservationWrapper
from mani_skill.utils.wrappers.record import RecordEpisode
from mani_skill.vector.wrappers.gymnasium import ManiSkillVectorEnv
from mani_skill.utils.registration import TimeLimitWrapper

/usr/local/lib/python3.10/dist-packages/sapien/__init__.py:2: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:3121: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:3121: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.cloud')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/usr/l

In [ ]:
num_files = 50 
num_steps = 80
num_envs = 80

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

checkpoint_path = "/kaggle/input/new-145-ckpt/ckpt_145.pt"
checkpoint = torch.load(checkpoint_path)

eval_output_dir = "./dataset_stackcube_v1"
env_kwargs = dict(obs_mode="rgb", render_mode='all', sim_backend="gpu")

eval_envs = gym.make('StackCube-v1', num_envs=num_envs, **env_kwargs)
eval_envs = TimeLimitWrapper(eval_envs, max_episode_steps=num_steps)
eval_envs = FlattenRGBDObservationWrapper(eval_envs, rgb=True, depth=False, state=True)

eval_envs = ManiSkillVectorEnv(eval_envs, num_envs=num_envs, ignore_terminations=True, record_metrics=True)

eval_obs, _ = eval_envs.reset()
agent = Agent(eval_envs, sample_obs=eval_obs).to(device)
agent.load_state_dict(checkpoint)

for i in tqdm(range(num_files)):
    eval_envs = RecordEpisode(
                          eval_envs, 
                          output_dir=eval_output_dir, 
                          save_trajectory=True, 
                          trajectory_name=f"trajectory_{i}", 
                          max_steps_per_video=num_steps, 
                          video_fps=30
    )
    obs, _ = eval_envs.reset()

    for _ in range(num_steps):
        with torch.no_grad():
            action = agent.get_action(obs, deterministic=True)
            obs, reward, terminated, truncated, info = eval_envs.step(action)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-3-e67b38c29d3c>:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by

Download complete.


/usr/local/lib/python3.10/dist-packages/torch/random.py:167: UserWarning: CUDA reports that you have 2 available devices, and you have used fork_rng without explicitly specifying which devices are being used. For safety, we initialize *every* CUDA device by default, which can be quite slow if you have a lot of CUDAs. If you know that you are only making use of a few CUDA devices, set the environment variable CUDA_VISIBLE_DEVICES or the 'devices' keyword argument of fork_rng with the set of devices you are actually using. For example, if you are using CPU only, set device.upper()_VISIBLE_DEVICES= or devices=[]; if you are using device 0 only, set CUDA_VISIBLE_DEVICES=0 or devices=[0].  To initialize all devices and suppress this warning, set the 'devices' keyword argument to `range(torch.cuda.device_count())`.
  warnings.warn(message)


  0%|          | 0/50 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/mani_skill/__init__.py:41: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  logger.warn("mani_skill is not installed with git.")
2025-01-14 11:53:54,867 - mani_skill  - WARNING - mani_skill is not installed with git.
/usr/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(
2025-01-14 11:55:33,209 - mani_skill  - WARNING - mani_skill is not installed with git.
2025-01-14 11:58:55,732 - mani_skill  - WARNING - mani_skill is not installed with git.
